In [ ]:
import os
import sys 

while "src" not in os.listdir(): os.chdir("..")

if "./src" not in sys.path: sys.path.append("./src")

In [ ]:
import json

from uuid import uuid4

from langchain_core.language_models.fake_chat_models import FakeMessagesListChatModel
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langchain_core.messages.tool import tool_call 

from langgraph.types import Command
from langgraph.checkpoint.memory import InMemorySaver

from smart_investigator.foundation.agents.ticketing_agent_graph import create_ticketing_agent
from smart_investigator.foundation.agents.travel_agent_graph import create_travel_agent
from smart_investigator.foundation.agents.travel_agent_tool_agents import create_ticketing_agent_tool

In [ ]:
def create_ai_message(msg:str) -> AIMessage:
    try:
        content = json.loads(msg)
        return AIMessage(content="calling a tool ... ", tool_calls=[tool_call(**tc) for tc in content])
    except:
        return AIMessage(content=msg)

class FakeMessagesListChatModelWithTools(FakeMessagesListChatModel):
    def bind_tools(self, tools:list):
        return self

In [ ]:
example_conversation = """
A: Hi, I am a ticketing agent. How can I help you?
Q: Hello, I want to go to Bali. Can you help me book a flight?
A: [{"id": "tkt_1", "name": "ask_for_help_tkt", "args": {"question": "From where, Human? Round-trip or one-way?"}}]
Q: From Sydney, Australia. Round-trip. 
A: [{"id": "tkt_2", "name": "check_ticket_price", "args": {"origin": "Sydney", "destination": "Bali", "round_trip": true}}, {"id": "tkt_3", "name": "ask_for_help_tkt", "args": {"question": "How many heads, Human?"}}]
Q: Just one. How much would it be?
A: [{"id": "tkt_4", "name": "calculate_total", "args": {"quantity": 1, "unit_price": 1000.99}}]
A: It will be $1000.99, ok to proceed?
Q: Yes, please book the flight.
A: Great! Your flight from Sydney to Bali has been booked. Safe travels!
Q: exit
"""

human_questions = [qim[3:] for qim in example_conversation.split("\n") if qim.startswith("Q: ") or len(qim.strip())==0]
ai_responses = [aim[3:] for aim in example_conversation.split("\n") if aim.startswith("A: ")]

responses=[create_ai_message(msg) for msg in ai_responses]

fake_model = FakeMessagesListChatModelWithTools(
    responses=responses
)

ticketing_agent = create_ticketing_agent(fake_model, InMemorySaver(), agent_tools=[])

input_iter = iter(human_questions)
config={"configurable": {"thread_id": str(uuid4()) }}
while True:    
    content = next(input_iter) # input("Your response (or 'exit' to quit): ")
    human_message = HumanMessage(content=content)
    human_message.pretty_print()
    if content.lower() in ['exit', 'quit']:
        AIMessage(content="Goodbye!").pretty_print()
        break
    response = ticketing_agent.invoke({"messages": [human_message], "loop_counter": 0}, config=config)
    while "__interrupt__" in response:
        print ("in interrupt")
        response["messages"][-1].pretty_print()
        human_response = next(input_iter) # input (str(response["messages"][-1].content) + "\nYour response: ")
        HumanMessage(content=human_response).pretty_print()
        response = ticketing_agent.invoke(Command(resume=response), config=config)

    response["messages"][-1].pretty_print()

In [ ]:
example_conversation = """
A: Hi, I am a ticketing agent. How can I help you?
Q: Hello, I want to go to Bali. Can you help me book a flight?
A: [{"id": "tkt_1", "name": "ask_for_help_tkt", "args": {"question": "From where, Human? Round-trip or one-way?"}}]
Q: From Sydney, Australia. Round-trip. 
A: [{"id": "tkt_2", "name": "check_ticket_price", "args": {"origin": "Sydney", "destination": "Bali", "round_trip": true}}]
A: [{"id": "tkt_3", "name": "ask_for_help_tkt", "args": {"question": "How many heads, Human?"}}]
Q: Just one. How much would it be?
A: [{"id": "tkt_4", "name": "calculate_total", "args": {"quantity": 1, "unit_price": 1000.99}}]
A: It will be $1000.99, ok to proceed?
Q: Yes, please book the flight.
A: [{"id": "tkt_5", "name": "book_ticket", "args": {"origin": "Sydney, AU", "destination": "Bali, DPS", "round_trip": true, "quantity": 1, "unit_price": 1000.99}}]
"""

human_questions = [qim[3:] for qim in example_conversation.split("\n") if qim.startswith("Q: ") or len(qim.strip())==0]
ai_responses = [aim[3:] for aim in example_conversation.split("\n") if aim.startswith("A: ")]

responses=[create_ai_message(msg) for msg in ai_responses]

fake_model = FakeMessagesListChatModelWithTools(
    responses=responses
)

ticketing_agent = create_ticketing_agent(fake_model, InMemorySaver(), agent_tools=[])

input_iter = iter(human_questions)
config={"configurable": {"thread_id": str(uuid4()) }}
response = {"messages": [], "loop_counter": 0, "ticket_booked": False}
while not response["ticket_booked"]: 
    
    content = next(input_iter) or "" # input("Your response (or 'exit' to quit): ")
    human_message = HumanMessage(content=content)
    human_message.pretty_print()
    response["messages"].append(human_message)
    response = ticketing_agent.invoke(response, config=config)

    while "__interrupt__" in response:
        print ("in interrupt")
        response["messages"][-1].pretty_print()
        human_response = next(input_iter) # input (str(response["messages"][-1].content) + "\nYour response: ")
        HumanMessage(content=human_response).pretty_print()
        response = ticketing_agent.invoke(Command(resume=response), config=config)
    
    response["messages"][-1].pretty_print()

In [ ]:
example_conversation = """
A: Hi, I am a travel agent. How can I help you?
Q: Hello, I want to eat Chicken Hor Fun. 
A: It seems you should visit Malaysia. Do you want me to help with your planning?
Q: Yes, please. Do I need to take my umbrella there?
A: [{"id": "travel_1", "name": "get_weather", "args": {"location": "Malaysia"}}]
A: The weather in Malaysia is sunny. No umbrella needed. May I transfer you to a ticketing agent for your flight booking?
Q: Yes, that would be great.
A: [{"id": "travel_2", "name": "ticketing_agent_tool", "args": {"query": "This person wants to go to KL, Malaysia. Please help book a flight."}}]
Q: From Sydney, Australia. Round-trip. # responding to interrupt
Q: Just one. How much would it be? # responding to interrupt
Q: Yes, please book the flight. # responding to interrupt
A: Ticketing said your ticket is booked. Is there anything else you want me to help you with?
Q: exit
"""

example_conversation_ticketing = """Q: This person wants to go to KL, Malaysia. Please help book a flight.
A: [{"id": "tkt_1", "name": "ask_for_help_tkt", "args": {"question": "From where, Caller? Round-trip or one-way?"}}]
A: [{"id": "tkt_2", "name": "check_ticket_price", "args": {"origin": "Sydney", "destination": "Kuala Lumpur", "round_trip": true}}]
A: [{"id": "tkt_3", "name": "ask_for_help_tkt", "args": {"question": "How many heads, Caller?"}}]
A: [{"id": "tkt_4", "name": "calculate_total", "args": {"quantity": 1, "unit_price": 1000.99}}]
A: [{"id": "tkt_5", "name": "ask_for_help_tkt", "args": {"It will be $1000.99, ok to proceed?"}}]
A: Great! Your flight from Sydney to Bali has been booked. Safe travels!
"""

human_questions = [qim[3:] for qim in example_conversation.split("\n") if qim.startswith("Q: ") or len(qim.strip())==0]
ai_responses = [aim[3:] for aim in example_conversation.split("\n") if aim.startswith("A: ")]

human_questions_ticketing = [qim[3:] for qim in example_conversation_ticketing.split("\n") if qim.startswith("Q: ") or len(qim.strip())==0]
ai_responses_ticketing = [aim[3:] for aim in example_conversation_ticketing.split("\n") if aim.startswith("A: ")]

######

responses_ticketing=[create_ai_message(msg) for msg in ai_responses_ticketing]

fake_model_ticketing = FakeMessagesListChatModelWithTools(
    responses=responses_ticketing
)

ticketing_agent = create_ticketing_agent(fake_model_ticketing, InMemorySaver(), agent_tools=[])

######

responses = [create_ai_message(msg) for msg in ai_responses]

fake_model = FakeMessagesListChatModelWithTools(responses=responses)

travel_agent = create_travel_agent(fake_model, InMemorySaver(), agent_tools=[create_ticketing_agent_tool(ticketing_agent)])


input_iter = iter(human_questions)
config={"configurable": {"thread_id": str(uuid4()) }}
response = {"messages": [], "loop_counter": 0}
while True: 
    
    content = next(input_iter) # input("Your response (or 'exit' to quit): ")
    human_message = HumanMessage(content=content)
    human_message.pretty_print()
    response["messages"].append(human_message)
    response = travel_agent.invoke(response, config=config)

    while "__interrupt__" in response:
        print ("in interrupt")
        response["messages"][-1].pretty_print()
        AIMessage(response["__interrupt__"][0].value).pretty_print()
        human_response = next(input_iter) # input (str(response["messages"][-1].content) + "\nYour response: ")
        HumanMessage(content=human_response).pretty_print()
        response = travel_agent.invoke(Command(resume=human_response), config=config)
    
    response["messages"][-1].pretty_print()